### Домашнее задание 2 - 10 баллов

В этом задании вам предстоит продолжить работу с датасетом lenta-ru-news для той же задачи - классификации текстов по топикам. Можно переиспользовать подготовленные данные из ДЗ 1 или загрузить их заново.

1. Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции 60/20/20. В качестве целевой переменной используйте атрибут `topic`
2. Обучите word2vec-эмбеддинги с помощью библиотеки gensim - **2 балла**
  - создайте модель для обучения на ваших данных, опишите, какими значениями вы инициализировали гиперпараметры модели, и почему
  - визуально оцените внутреннее (intrinsic) качество получившихся эмбеддингов, используя методы gensim - doesnt_match, most_similar
3. Загрузите предобученные эмбеддинги из navec и rusvectores (на ваш вкус) - **1 балл**
4. Обучите модель `sklearn.linear_model.LogisticRegression` с тремя вариантами векторизации текстов и сравните их качество между собой на валидационной выборке: **2 балла**
  - ваши эмбеддинги w2v
  - предобученные эмбеддинги navec
  - предобученные эмбеддинги rusvectores
5. Попробуйте улучшить качество модели, взяв для ее обучения лучший набор эмбеддингов и используя его с взвешиванием через tf-idf. То есть, необходимо каждый текст представить в виде взвешенного усреднения эмбеддингов его слов, где весами являются соответствующие коэффициенты tf-idf - **2 балла**
6. Финально сравните качество всех моделей на тестовой выборке - **1 балл**


**Общее**

- Принимаемые решения обоснованы (почему выбрана определенная архитектура/гиперпараметр/оптимизатор/преобразование и т.п.) - **1 балл**
- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **1 балл**

In [52]:
import pandas as pd
import numpy as np
import gensim.models
import spacy

from sklearn.model_selection import train_test_split
from navec import Navec
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

tqdm.pandas()

# 1. Загрузка данных

In [3]:
# загружу уже предобработанные тексты из hw_1
df = pd.read_csv('data/clean_lenta-ru-news.csv')
df.head()

,title,text,topic,processed_text,lemmatized_text
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,вице премьер социальным вопросам татьяна голик...,вица премьер социальный вопрос татьяна голиков...
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,австрийские правоохранительные органы представ...,австрийский правоохранительный орган представл...
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,сотрудники социальной сети проанализировали по...,сотрудник социальный сеть проанализировать пос...
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,начала расследования российского вмешательства...,начинать расследование российский вмешательств...
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,хакерская группировка опубликовала новые докум...,хакерский группировка опубликовывать новый док...


In [4]:
df.topic.value_counts()

topic
Россия               15151
Мир                  14421
Спорт                10045
Экономика             7682
Интернет и СМИ        6935
Силовые структуры     6925
Бывший СССР           6810
Культура              6578
Наука и техника       5645
Из жизни              4903
Ценности              4480
Дом                   3408
Путешествия           3223
Бизнес                1993
69-я параллель         815
Крым                   661
Культпросвет           307
Оружие                   1
Name: count, dtype: int64

In [ ]:
# у меня получилось так, что topic Оружие встретился всего 1 раз,
# поэтому пришлось его удалить из-за невозможности стратификации
df = df[df['topic'] != 'Оружие']

df = df.dropna()
X = df['lemmatized_text'].str.split()
y = df['topic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)
X_train.shape, X_val.shape, X_test.shape

((59989,), (19996,), (19997,))

# 2. Обучение W2V эмбеддингов с gensim

In [ ]:
model = gensim.models.Word2Vec(
    sentences=X_train,
    vector_size=300,  # потому что датасет не большой и эмбеддинги длиннее могут не выучиться
    window=5,  # максимальное расстояние между текущим и предсказанным словом, просто взяла base значение
    min_count=10,  # чтобы слово встречалось хотя бы 10 раз, можно поставить и 50
    negative=10,  # эпох не очень много, поэтому поставила не 5, чтобы как можно быльше слов при обучении как-то изменялись
    epochs=25,
    seed=42,
)

model.wv.most_similar(positive=['кошка'], topn=5)

[('питомец', 0.6081347465515137),
 ('кот', 0.5931268334388733),
 ('собака', 0.5848177671432495),
 ('котенок', 0.5780510902404785),
 ('кролик', 0.5720377564430237)]

In [41]:
model.wv.doesnt_match(['азия', 'европа', 'америка', 'бразилия', 'кролик'])

'кролик'

Визуально качество неплохое, общие зависимости слов улавливает

# 3. Загрузка векторов navec/rusvectors

In [ ]:
# !curl -L -o data/navec_news_v1_1B_250K_300d_100q.tar https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 25.4M    0  207k    0     0   151k      0  0:02:52  0:00:01  0:02:51  151k
  4 25.4M    4 1087k    0     0   455k      0  0:00:57  0:00:02  0:00:55  455k
  7 25.4M    7 1986k    0     0   599k      0  0:00:43  0:00:03  0:00:40  599k
 12 25.4M   12 3364k    0     0   782k      0  0:00:33  0:00:04  0:00:29  782k
 18 25.4M   18 4925k    0     0   928k      0  0:00:28  0:00:05  0:00:23  992k
 26 25.4M   26 6913k    0     0  1096k      0  0:00:23  0:00:06  0:00:17 1360k
 35 25.4M   35 9129k    0     0  1250k      0  0:00:20  0:00:07  0:00:13 1636k
 44 25.4M   44 11.2M    0     0  1389k      0  0:00:18  0:00:08  0:00:10 1915k
 52 25.4M   52 13.4M    0     0  1478k      0  0:00

In [31]:
# !curl -L -o data/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz https://rusvectores.org/static/models/rusvectores4/ruwikiruscorpora/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  376M    0 1156k    0     0  3516k      0  0:01:49 --:--:--  0:01:49 3525k
  2  376M    2 11.2M    0     0  8694k      0  0:00:44  0:00:01  0:00:43 8696k
  5  376M    5 20.8M    0     0  9169k      0  0:00:42  0:00:02  0:00:40 9172k
  8  376M    8 31.1M    0     0  9598k      0  0:00:40  0:00:03  0:00:37 9600k
 11  376M   11 41.5M    0     0  9841k      0  0:00:39  0:00:04  0:00:35 9844k
 13  376M   13 52.1M    0     0   9.7M      0  0:00:38  0:00:05  0:00:33 10.1M
 16  376M   16 62.7M    0     0   9.9M      0  0:00:37  0:00:06  0:00:31 10.2M
 19  376M   19 73.1M    0     0   9.9M      0  0:00:37  0:00:07  0:00:30 10.4M
 21  376M   21 81.9M    0     0   9.8M      0  0:00:38  0:00:08  0:00:30 10.1M
 24  376M   24 91.9M    0     0   9.8M      0  0:00

In [ ]:
navec_model = Navec.load('data/navec_news_v1_1B_250K_300d_100q.tar')
rusvectores_model = gensim.models.KeyedVectors.load_word2vec_format(
    'data/ruwikiruscorpora_upos_skipgram_300_2_2018.vec.gz'
)

In [43]:
def text_to_vector(texts: list[list], model, dim: int) -> np.array:
    vectors = []
    for text in texts:
        vectors.append(
            np.mean([model[word] for word in text if word in model], axis=0)
            if any(word in model for word in text)
            else np.zeros(dim)
        )
    return np.array(vectors)


train_w2v = text_to_vector(X_train, model.wv, 256)
val_w2v = text_to_vector(X_val, model.wv, 256)
test_w2v = text_to_vector(X_test, model.wv, 256)

train_navec = text_to_vector(X_train, navec_model, 300)
val_navec = text_to_vector(X_val, navec_model, 300)
test_navec = text_to_vector(X_test, navec_model, 300)

In [ ]:
nlp = spacy.load('ru_core_news_sm', enable=['tok2vec', 'morphologizer', 'lemmatizer'])


def add_pos_tag(texts: list[list]) -> list[list]:
    pos_texts = nlp.pipe([' '.join(text) for text in texts], batch_size=128, n_process=-1)
    return [[f'{token.lemma_}_{token.pos_}' for token in text] for text in pos_texts]


train_rusvectores = add_pos_tag(X_train)
val_rusvectores = add_pos_tag(X_val)
test_rusvectores = add_pos_tag(X_test)

train_rusvectores = text_to_vector(train_rusvectores, rusvectores_model, 300)
val_rusvectores = text_to_vector(val_rusvectores, rusvectores_model, 300)
test_rusvectores = text_to_vector(test_rusvectores, rusvectores_model, 300)

# 4. Обучение логрегрессии

In [60]:
def train_and_evaluate(X_train, X_val, y_train, y_val) -> float:
    clf = LogisticRegression(max_iter=1000, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    return accuracy_score(y_val, y_pred)


acc_w2v = train_and_evaluate(train_w2v, val_w2v, y_train, y_val)
acc_navec = train_and_evaluate(train_navec, val_navec, y_train, y_val)
acc_rusv = train_and_evaluate(train_rusvectores, val_rusvectores, y_train, y_val)

print(f'Accuracy векторов на обученном W2V: {acc_w2v:.4f}')
print(f'Accuracy векторов Navec: {acc_navec:.4f}')
print(f'Accuracy векторов RucVectores: {acc_rusv:.4f}')

Accuracy векторов на обученном W2V: 0.8106
Accuracy векторов Navec: 0.7928
Accuracy векторов RucVectores: 0.7264


In [53]:
X_train

70753    [депутат, верховный, рад, надежда, савченко, з...
88036    [управление, росреестр, московский, область, ф...
51971    [заявка, сборная, нигер, юношеский, чемпионат,...
96472    [кандидат, президент, франция, бывший, премьер...
13644    [ученый, гонконгский, баптистский, университет...
                               ...                        
2934     [любительница, лотерея, американский, штат, ми...
97888    [исследователь, медицинский, школа, вашингтонс...
50026    [квартира, речной, улица, подмосковный, балаши...
45941    [исследователь, манитобский, университет, кана...
77912    [стоимость, самый, дорогой, загородный, домовл...
Name: lemmatized_text, Length: 59989, dtype: object

# 5. Логрегрессия с TF-IDF взвешиванием

In [57]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
vectorizer.fit(X_train)
tfidf_weights = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))

In [59]:
def weighted_text_to_vector(texts: list[list], model, dim: int, tfidf_weights) -> np.array:
    vectors = []
    for text in texts:
        word_vectors, weights = [], []
        for word in text:
            if word in model and word in tfidf_weights:
                word_vectors.append(model[word] * tfidf_weights[word])
                weights.append(tfidf_weights[word])
        vectors.append(np.average(word_vectors, axis=0, weights=weights) if word_vectors else np.zeros(dim))
    return np.array(vectors)


train_weighted = weighted_text_to_vector(X_train, model.wv, 300, tfidf_weights)
val_weighted = weighted_text_to_vector(X_val, model.wv, 300, tfidf_weights)
test_weighted = weighted_text_to_vector(X_test, model.wv, 300, tfidf_weights)

acc_weighted = train_and_evaluate(train_weighted, val_weighted, y_train, y_val)

print(f'Accuracy модели на взвешенных векторах обученного W2V: {acc_weighted:.4f}')


Accuracy модели на взвешенных векторах обученного W2V: 0.7990


Интересно, что качество только упало

# 6. Финальное сравнение на тесте

In [ ]:
acc_w2v = train_and_evaluate(train_w2v, test_w2v, y_train, y_test)
acc_navec = train_and_evaluate(train_navec, test_navec, y_train, y_test)
acc_rusv = train_and_evaluate(train_rusvectores, test_rusvectores, y_train, y_test)
acc_weighted_w2v = train_and_evaluate(train_weighted, test_weighted, y_train, y_test)

print(f'Accuracy векторов на обученном W2V: {acc_w2v:.4f}')
print(f'Accuracy взвешенных векторов на обученном W2V: {acc_weighted_w2v:.4f}')
print(f'Accuracy векторов Navec: {acc_navec:.4f}')
print(f'Accuracy векторов RucVectores: {acc_rusv:.4f}')

Accuracy векторов на обученном W2V: 0.8007
Accuracy взвешенных векторов на обученном W2V: 0.7902
Accuracy векторов Navec: 0.7832
Accuracy векторов RucVectores: 0.7237
